In [ ]:
PROJECT_ID = "praxis-granite-420515"  # @param {type:"string"}
REGION = "us-central1"  # @param {type: "string"}
BUCKET_URI = "gs://praxis-granite-420515-experiments-staging-bucket"  # @param {type:"string"}
EXPERIMENT_NAME = "experiments-tutorial"  # @param {type:"string"}

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform
! pip3 install --upgrade --quiet google-cloud-aiplatform[autologging]

! gcloud config set project {PROJECT_ID}

# Only if your bucket doesn't already exist: Uncomment the following code to create your Cloud Storage bucket.
# ! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

In [ ]:
from google.cloud import aiplatform
aiplatform.init(
    project=PROJECT_ID,
    staging_bucket=BUCKET_URI,
    location=REGION,
    experiment=EXPERIMENT_NAME)

aiplatform.autolog()

In [ ]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models
import vertexai
from vertexai.generative_models import GenerativeModel, ChatSession
vertexai.init(project="praxis-granite-420515", location="us-central1")
model=GenerativeModel(model_name="gemini-1.0-pro",system_instruction=" You are an intelligent programmer. You must complete the python function given to you by the user. And you must follow the format they present when giving your answer!"
)

In [ ]:
!conda create -n codex python=3.7
! conda activate codex
! git clone https://github.com/openai/human-eval
! pip install -e human-eval
# Set the working directory
%cd /content/human-eval
!pip install jsonlines

In [ ]:
generation_config = {
    "temperature": 0.0,
}


In [ ]:
from human_eval.data import write_jsonl, read_problems
import pandas as pd
import time
import jsonlines
def generate_one_completion(prompt):
  prompt_msg =  "You are an intelligent programmer. You must complete the python function given to you by the user. No explanation of function required And you must follow the format they present when giving your answer!Exclude examples"+prompt
  response = model.generate_content(prompt_msg,generation_config=generation_config)
  return response.candidates[0].text



In [ ]:
import pandas as pd
import time
import jsonlines
import numpy as np

if __name__ == '__main__':
    jsonObj = pd.read_json(path_or_buf="https://github.com/openai/human-eval/raw/master/data/HumanEval.jsonl.gz", lines=True)
    command = 0
    i = 0
    durationTimes = []
    with jsonlines.open('Gemini1.0-pro.jsonl', mode='w') as writer:
        while i < 164:
            if command < 4:
                start_time = time.time()
                response = generate_one_completion(jsonObj["prompt"][i])
                duration = time.time() - start_time

                durationTimes.append(duration)
                response = response.replace("```python\n", "")
                response = response.replace("\n```", "")

                completion = {'task_id': jsonObj["task_id"][i], 'completion': response}
                writer.write(completion)
                command = command + 1
                i = i + 1
            else:
                command = 0
                time.sleep(90)
    print(np.sum(duration))
    print(np.average(duration))

1.478311538696289
1.478311538696289


In [98]:
!python3 human_eval/evaluate_functional_correctness.py Gemini1.0-pro.jsonl

Reading samples...
164it [00:00, 14632.95it/s]
Running test suites...
100% 164/164 [00:03<00:00, 49.40it/s]
Writing results to Gemini1.0-pro.jsonl_results.jsonl...
100% 164/164 [00:00<00:00, 26120.83it/s]
{'pass@1': 0.4146341463414634}
